1. pivot talbe of user movie table
2. calculate the weights between users
3. calculate the rating for i users

In [1]:
import pandas as pd
import numpy as np

In [2]:
rating = pd.read_csv('rating.csv', index_col=0)

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# link = pd.read_csv('link.csv', index_col=0)
# movie = pd.read_csv('movie.csv', index_col=0)
# genome_tags = pd.read_csv('genome_tags.csv', index_col=0)

In [4]:
rating.head()

,movieId,rating,timestamp
userId,,,
1,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
1,32,3.5,2005-04-02 23:33:39
1,47,3.5,2005-04-02 23:32:07
1,50,3.5,2005-04-02 23:29:40


In [5]:
flattened = pd.DataFrame(rating.to_records())

In [6]:
flattened.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
sample = flattened.head(100000)

In [8]:
user_movie_table = pd.pivot_table(sample, index=['userId'], columns=['movieId'])

In [9]:
user_movie_table.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          NaN    3.5    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          NaN    NaN    4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    4.0   
5          NaN    3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ...                                                                   \
movieId  ...   117511 117590 118354 118696 118900 118997 119141 125916 128488   
userId   ...                                                                    
1        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 128594  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  

[5 rows x 8227 columns]

In [10]:
#find two users pearson correlation

In [11]:
def weight_calculator(user_movie_table, index1, index2):
    """find pearson corr between two users
    Parameters
    --------------
    user_movie_table : pd.DataFrame
        user_movie rating info
    index1, index2 : Integer
        pointer in table
    """
    mu = user_movie_table.T
    user1 = mu[index1]
    user2 = mu[index2]
    common = mu[[index1,index2]].dropna()
    user1_watched = user1.dropna()
    user2_watched = user2.dropna()
    
    x1_mean = user1.mean()
    x2_mean = user2.mean()
    numerator = np.array(list(map(lambda x,y : (x-x1_mean)*(x-x2_mean), common[index1],common[index2]))).sum()
    
    left_corner = np.array(list(map(lambda x : (x-x1_mean)**2, user1_watched))).sum()**0.5
    right_corner = np.array(list(map(lambda x : (x-x2_mean)**2, user2_watched))).sum()**0.5
    denominator = left_corner * right_corner
    
    weight_between_two_users = numerator/denominator
    
    return weight_between_two_users

In [12]:
w = weight_calculator(user_movie_table, 1, 2)

In [13]:
w

0.014944306957911126

### => Note! I found the weight for a person to himself is not 1!

# Now, take the user1 as the target. Let's say I want to know whether I should recommend movie1 to him.

user3,6,8,10 has review for movie1

In [14]:
#1.find people who rates movie3

In [15]:
user_movie_table.ix[1,1]
#first index is row name, second index is index of column

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


3.5

In [16]:
movie3_rating = user_movie_table.ix[:,2]
movie3_rating[:10]

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


userId
1     NaN
2     4.0
3     NaN
4     NaN
5     NaN
6     3.0
7     3.0
8     5.0
9     NaN
10    NaN
Name: (rating, 3), dtype: float64

In [17]:
#so I find the movie3 all user ratings
movie3_rating = movie3_rating.dropna()
movie3_rating[:10]

userId
2     4.0
6     3.0
7     3.0
8     5.0
12    3.0
19    4.0
38    3.0
75    3.0
80    3.0
91    3.0
Name: (rating, 3), dtype: float64

In [18]:
#get users who have rated movie3
users_movie3 = list(pd.DataFrame(movie3_rating.reset_index())['userId'])
users_movie3[:10]

[2, 6, 7, 8, 12, 19, 38, 75, 80, 91]

In [19]:
movie3_rating_score = list(pd.DataFrame(movie3_rating.reset_index())['rating'][3])
movie3_rating_score[:10]

[4.0, 3.0, 3.0, 5.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0]

In [20]:
mu = user_movie_table.T

In [21]:
#calculate all weight between user1 and all users_movie3
numerator_list = []
denominator_list = []
for i in range(len(users_movie3)):
    weight_1_i = weight_calculator(user_movie_table, 1, users_movie3[i])
    
    useri_watched = mu[users_movie3[i]].dropna()
    xi_mean = useri_watched.mean()
    
    score_i = movie3_rating_score[i]
    numerator_i = weight_1_i * (score_i - xi_mean)
    numerator_list.append(numerator_i)
    
    denominator_i = (weight_1_i**2)**0.5
    denominator_list.append(denominator_i)
    

In [22]:
numerator = np.array(numerator_list).sum()
denominator = np.array(denominator_list).sum()

In [23]:
user1_watched = mu[1].dropna()
x1_mean = user1_watched.mean()

In [24]:
socre_1_3 = x1_mean + numerator/denominator

In [25]:
socre_1_3

3.371630498700262